In [2]:
import os
from pathlib import Path

In [27]:
atom_names = ["O", "C", "H", "H", "H", "H"]

In [28]:
com_file_template = """%nproc=4
%mem=5760MB
%chk={KEY}.chk
#P PBE1PBE/aug-cc-pVDZ scf(maxcycle=200) 

Gaussian input

0 1
{ATOMS}
"""

In [29]:
print(com_file_template.format(ATOMS="...", KEY="..."))

%nproc=4
%mem=5760MB
%chk=....chk
#P PBE1PBE/aug-cc-pVDZ scf(maxcycle=200) 

Gaussian input

0 1
...



In [30]:
job_file_template = """#!/bin/bash

#SBATCH --job-name={KEY}
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=1600MB
#SBATCH --partition=vshort
#SBATCH --exclude=node[109-124]
#$ -S /bin/bash

# Define Parameters
hostname
export WORKDIR=/home/boittier/methanol_dih/input
export DATADIR=/home/boittier/methanol_dih/data
export INPFILE={KEY}.com
export OUTFILE={KEY}.out
export CHKFILE={KEY}.chk
export FCKFILE={KEY}.fchk
export DNSFILE={KEY}.d.cube
export ESPFILE={KEY}.p.cube

# Make scratch temporary directory
export STMPDIR=/scratch/$USER/jobs/tmp.$SLURM_JOBID
if [ -d $STMPDIR ]; then
  echo "$STMPDIR exists; double job start; exit"
  exit 1
fi
mkdir -p $STMPDIR

cd $STMPDIR

# Copy input file
cp $WORKDIR/$INPFILE $STMPDIR/$INPFILE

# Prepare Gaussian parameters
export g16root=/opt/cluster/programs/g16-c.01
source $g16root/g16/bsd/g16.profile
export GAUSS_SCRDIR=$STMPDIR

# Execute Gaussian jobs
$g16root/g16/g16 $STMPDIR/$INPFILE $STMPDIR/$OUTFILE
    
# Copy result file to output directory
cp $STMPDIR/$OUTFILE $DATADIR/$OUTFILE

# Delete output file if successfully copied
status=$?
if [ $status -eq 1 ]; then
    echo "$STMPDIR/$OUTFILE could not be transferred to $OUTPDIR/$OUTFILE"
fi
        
# Write formchk file
$g16root/g16/formchk $CHKFILE $FCKFILE

# Extract density cube file
$g16root/g16/cubegen 0 Density $FCKFILE $DNSFILE -2 h

# Extract esp cube file
$g16root/g16/cubegen 0 Potential $FCKFILE $ESPFILE -2 h

# Copy result to data directory
cp $CHKFILE $DNSFILE $ESPFILE $DATADIR/.

if [ $status -eq 0 ] 
then 
    # Delete temporary directory
    rm -rf $STMPDIR
else
    echo "$STMPDIR results could not be transferred to $DATADIR"
fi

"""

In [31]:
xyz_path = Path("/home/boittier/pcbach/param2/methanol/kern/rkkk/t298c/gas/test.xyz")


In [32]:
xyz_file_lines = open(xyz_path).readlines()

In [46]:
for i in range(1000):
    KEY = f"meoh_{i}"
    test_xyz = xyz_file_lines[i*18:i*18+18]
    atom_str = ""
    for i, _ in enumerate(test_xyz[2:8]):
        coords = " ".join(_.split()[1:])
        atom_str+= (f"{atom_names[i]} {coords}\n")
    comfile = com_file_template.format(ATOMS=atom_str, KEY=KEY)
    with open(f"comfiles/{KEY}.com", "w") as f:
        f.write(comfile)
    jobfile = job_file_template.format(KEY=KEY)
    with open(f"comfiles/{KEY}.sh", "w") as f:
        f.write(jobfile)

In [44]:

# print(atom_str)

In [40]:
print(com_file_template.format(ATOMS=atom_str, KEY="meoh_0"))

%nproc=4
%mem=5760MB
%chk=meoh_0.chk
#P PBE1PBE/aug-cc-pVDZ scf(maxcycle=200) 

Gaussian input

0 1
O 1.4200 0.0000 0.0000
C 0.0000 0.0000 0.0000
H 1.7467 0.9240 0.0000
H -0.3633 -1.0277 0.0000
H -0.3633 0.5138 0.8900
H -0.3633 0.5138 -0.8900




In [43]:
print(job_file_template.format(KEY="meoh_0"))

#!/bin/bash

#SBATCH --job-name=meoh_0
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=1600MB
#SBATCH --partition=vshort
#SBATCH --exclude=node[109-124]
#$ -S /bin/bash

# Define Parameters
hostname
export WORKDIR=/home/boittier/methanol_dih/input
export DATADIR=/home/boittier/methanol_dih/data
export INPFILE=meoh_0.com
export OUTFILE=meoh_0.out
export CHKFILE=meoh_0.chk
export FCKFILE=meoh_0.fchk
export DNSFILE=meoh_0.d.cube
export ESPFILE=meoh_0.p.cube

# Make scratch temporary directory
export STMPDIR=/scratch/$USER/jobs/tmp.$SLURM_JOBID
if [ -d $STMPDIR ]; then
  echo "$STMPDIR exists; double job start; exit"
  exit 1
fi
mkdir -p $STMPDIR

cd $STMPDIR

# Copy input file
cp $WORKDIR/$INPFILE $STMPDIR/$INPFILE

# Prepare Gaussian parameters
export g16root=/opt/cluster/programs/g16-c.01
source $g16root/g16/bsd/g16.profile
export GAUSS_SCRDIR=$STMPDIR

# Execute Gaussian jobs
$g16root/g16/g16 $STMPDIR/$INPFILE $STMPDIR/$OUTFILE
    
# Copy result fi

In [13]:
i = 2


['   16\n',
 '   dumped from DCM module\n',
 'C    1.4498   -0.0160   -0.0001\n',
 'C    0.0016    0.0074    0.0001\n',
 'C    1.7248    0.9247    0.0000\n',
 'C   -0.3638   -1.0196   -0.0000\n',
 'C   -0.3679    0.5138    0.8918\n',
 'C   -0.3678    0.5136   -0.8918\n',
 'O    1.6888    0.7610    0.1662   -0.5952\n',
 'O    1.7459    0.8418    0.0777    1.0000\n',
 'O    1.5075   -0.1516   -0.1677   -0.1280\n',
 'O    1.4465   -0.0744    0.1673   -0.6621\n',
 'O    1.4268    0.0359   -0.1676   -0.7297\n',
 'O    1.2430   -0.1287   -0.0134    1.0000\n',
 'O    0.1118   -0.1600    0.0430    0.0380\n',
 'O   -0.3172    0.5781    1.0206    0.0263\n',
 'O   -0.3094    0.5715   -1.0084    0.0263\n',
 'O   -0.3560   -1.2281    0.0129    0.0263\n']